# Multiple EM for Motif Elicitation
**Author:** Lauren Enriquez

In [1]:
import numpy as np
from sklearn.preprocessing import normalize
import string
from Bio import Entrez
from Bio import SeqIO
from Bio import SeqFeature
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import collections

In [2]:
# Constant used in this exercise
# Fill in all of the ...s/TODOs
width = 8

# Helper dict for indexing
let_dict = {"A":0, "C":1, "G":2, "T":3}

## 1. Read in Fasta Sequences

In [3]:
# Read in the files using BioPython
# TODO:
fastas = list(SeqIO.parse("motif-regions.fa", "fasta"))

In [4]:
# Extract strings of sequences from the above files
# TODO:
sequences = [str(i.seq) for i in fastas]

## 2. Create p_0

In [5]:
# Initialize p with a uniform background

def init_p(l, w, seqs, let):
    p = np.zeros((4, w+1))

    # set a uniform background for each A C G T
    # TODO:
    for i in range(0,4):
        
        # TODO:
        p[i][0] = 0.25

    # set motif positions
    for i in range(l-w+1):
        for sequence in seqs:
            for j in range(w):
                
                # Fill in p_0
                # TODO:
                p[let[sequence[i+j]]][j+1] +=1
                
    # normalize columns to sum to 1
    p = normalize(p, axis = 0, norm = 'l1')
    
    return p

## 3. Fill in EM iteration

In [6]:
# Define a general function to run EM

def run_EM(w, seqs, let, init_p, up_prob, up_motif, epsilon = 2**-64):
    
    l = len(seqs[0])
    
    no_change = False
    
    # set an initial p_t_1
    # TODO:
    p_t_1 = init_p(l, w, seqs, let)
        
    while not no_change:
        
        # Label the following steps as E step or M step in the comment preceding
        
        # TODO:
        # E step
                
        z_t = up_motif(l, w, p_t_1, seqs, let)
        
        # TODO:
        # M step
                
        p_t = up_prob(l, w, z_t, seqs, let)
        
        diff = np.subtract(p_t, p_t_1)
        
        # Write a condition to stop the EM iterations (use epsilon and diff)
        # TODO:
        if np.max(diff) < epsilon:        
            no_change = True
        else:
            # Update p_t_1
            # TODO:
            p_t_1 = p_t
                
    return p_t, z_t

## 4. Fill in function to update z_t

In [7]:
# Define a function to update z

def up_motif(l, w, p_t_1, seqs, let):
    z_t = np.zeros((len(seqs), l-w+1))

    for i, sequence in enumerate(seqs):
        for j in range(l-w+1):
            
            # Fill in z_t using p_t_1
            # Ignore background as we're assuming 0.25 for all 4
            # TODO:
            motif = sequence[j:j+w]
            p_vals = [p_t_1[let[motif[k]]] for k in range(w)]
            z_t[i][j] = np.prod(p_vals)
    
    # Normalize z_t
    # TODO:
    z_t = normalize(z_t, axis = 1, norm = 'l1')
        
    return z_t
    

## 5. Fill in function to update p_t

In [8]:
# Define a function to update p

def up_prob(l, w, z_t, seqs, let):
    p_t = np.zeros((4, w+1))
    
    n = np.zeros((4, w+1))
    
    # Fill in n for k > 0
    for k in range(1, w+1):
        for letter in let.keys():
            sum_z = 0
            for i, sequence in enumerate(seqs):
                
                # Write j_vals according to the condition seen in lecture
                # TODO:
                j_vals = k
                      
                # Add to the sum using z_t
                # TODO:
                sum_z += z_t[i][j_vals]
            
            # Fill in the correct indices
            # TODO:
            n[let[letter]][k] = sum_z
                 
    # Fill in n for k == 0
    
    # May help to make the next step easier
    joined_seq = "".join(seqs)
    
    # Create a dict with total counts of A,C,G,T
    # TODO:
    counts = {"A":joined_seq.count("A"),
              "C":joined_seq.count("C"),
              "G":joined_seq.count("G"),
              "T":joined_seq.count("T")}
        
    # Sum across the rows of n
    # TODO:
    sum_n_j = np.sum(n,axis = 1) 
        
    for letter in let.keys():
        
        # Fill in the correct indices and its value
        # TODO:
        n[let[letter]][0] = counts[letter] - sum_n_j[let[letter]]

    
    # Use n to fill in p_t
    # Pseudo-count = 1
    # TODO:
    sum_n_b_k = (n.sum()) + 4
    for k in range(w+1):
        for letter in let.keys():
            x = (n[let[letter]][k]) + 1
            y = x / sum_n_b_k
            p_t[let[letter]][k] = y
            
    return p_t

## 6. Run the EM to find the final p and z

In [9]:
# Use the variables set at the start and 
# TODO:
p_end, z_end = run_EM(width, sequences, let_dict, init_p, up_prob, up_motif, epsilon = 2**-64)
print(p_end.shape, z_end.shape)

(4, 9) (50, 94)


## 7. Determine Motifs

In [10]:
# Find the indices of the max element for each row in z_end
# TODO:
motif_indices = np.argmax(z_end, 1)
print(motif_indices,"\n")

# Get the 'width' characters long motifs using seqs
# TODO:
motifs = [sequences[i][motif_indices[i]:motif_indices[i]+8] for i in range(len(sequences))]
print(motifs)

[44 22 81  1 36  1 50 57 20 38 13 83 77 50 44 62 45 73 44 64 70  6 45 29
 16 36 36 85 78  3 23 85 17 78 75 63 46 13 80 50 39 63  8 49 63 11 34 82
 29 59] 

['AAAAAAAA', 'TGTTTATT', 'TAATACAA', 'TAAAGTTA', 'AAAAAACT', 'AGAAAAAA', 'AACAATTT', 'GAAAAAAA', 'AATAAATT', 'AAAAAATT', 'AAAAAAAA', 'AAAAAAAA', 'AAAAAAAT', 'AAACAATA', 'TTAAAAGT', 'AAATTTTA', 'TAAGAAAA', 'AAGAAAAA', 'TATATATA', 'AAAAAAAA', 'ATATACAA', 'ATATAGAA', 'TTTTAATA', 'TTTTGTAA', 'AAAAAAAA', 'AACAAAAT', 'AAAAATTT', 'AAAAAAAA', 'AAAAGAAT', 'ATACATAA', 'AAAAATAT', 'GAAAATTT', 'AAAATTAA', 'TTATAAAA', 'GAAAAAAA', 'AAGAAATT', 'AAAAAGAA', 'AATAACAA', 'AATAGAAT', 'GAAAAATT', 'AAAAAAAA', 'AATAATAA', 'AAAACTAT', 'TATTTTTA', 'GAAAAATA', 'AAAAATAA', 'GAAAATCT', 'AATTTTAA', 'AAAAAAAT', 'TTTAACTA']
